# Backbone Feature Detector - Darknet53
This will be an implementation of the backbonepart of Yolov3. The various comment will guide you into the code, but for a more general understanding of the project i suggest to read the README.md

## Importing the main libraries

In [9]:
# Utility libraries
import os
import random
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

# Core libraries
import numpy as np
import tensorflow as tf

from tensorflow import Tensor
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Conv2D, LeakyReLU, Add, BatchNormalization, GlobalAveragePooling2D, Dense, Softmax, Input


In [10]:
# Load the TensorBoard notebook extension (for visualization purposes)
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Model designing

### Residual Blocks
First of all, we need to write down the code for the residual blocks used in the Darknet-53, which is a 53 layer deep feature extractor used as a backbone for YOLOv3.

![Darknet_Architecture](img/darknet5353.jpg)


In [11]:
def Conv2D_plus(inputs, filters, kernel_size, stride = 1) -> Tensor:
    X = Conv2D(filters = filters,
                   kernel_size = kernel_size,
                   strides = stride,
                   padding = "same",
                   use_bias = False)(inputs)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.1)(X)
    return X

def ResidualUnit(inputs, filters_alpha, filters_beta) -> Tensor:
    Y = Conv2D_plus(inputs, filters_alpha, 1)
    Y = Conv2D_plus(Y, filters_beta, 3)
    Y = Add()([Y, inputs])
    return Y

def ResidualBlock(inputs, num_filters, num_blocks) -> Tensor:
    W = Conv2D_plus(inputs, num_filters, 3, stride = 2)
    for _ in range(num_blocks):
        W = ResidualUnit(W, num_filters // 2, num_filters)
    return W

## Defining the model itself


In [27]:
def Darknet(inputs, classification = False, num_classes = 10):
    Z = Conv2D_plus(inputs, 32, 3)
    Z = ResidualBlock(Z, 64, 1)
    Z = ResidualBlock(Z, 128, 2)
    Z = ResidualBlock(Z, 256, 8)
    Z = ResidualBlock(Z, 512, 8)
    Z = ResidualBlock(Z, 1024, 4)
    if classification:
        Z = GlobalAveragePooling2D()(Z)
        Z = Dense(num_classes)(Z)
        Z = Softmax()(Z)
    darknet = Model(inputs=inputs, outputs=Z, name="Darknet53")
    return darknet

inputs = Input(shape=(452, 602, 3))
model = Darknet(inputs)
# model.summary()

Now i wanna see how the weights are stored

In [20]:
for i in range(4):
    print(model.get_layer(index = i))
    try:
        model.get_layer(index = i).get_weights().shape
    except:
        for l in model.get_layer(index = i).get_weights(): print(l.shape)

(3, 3, 3, 32)
(32,)
(32,)
(32,)
(32,)


### Testing a dummy input with random weights

In [21]:
def get_test_input():
    img_ = tf.io.read_file("dog-cycle-car.png")
    img_ = tf.image.decode_png(img_, channels=3)
    img_ = tf.image.convert_image_dtype(img_, dtype=tf.float32)
    img_ = tf.reshape(img_, (1, 452, 602, 3))
    return img_

inp = get_test_input()
pred = model.predict(inp)
print(pred.shape)

(1, 15, 19, 1024)


## Loading weights
It may be ok not to load any weight if you look at https://arxiv.org/pdf/1811.08883.pdf.

Those are the weights obtained after training the darknet53 on the ImageNet Dataset, not the final trained ones. At this point i'm still not sure if they will be trainable in the "fine training" phase, we'll see.

In [31]:
weights_path = Path('/home/andrea/AI/ispr_yolo/weights/darknet_pretrained')
fp = weights_path.joinpath('darknet53.weights')
weights_array = np.fromfile(fp, dtype = np.float32, offset = 20)
weights_num_counter = 0

def swapPositions(list, pos1, pos2): 
    list[pos1], list[pos2] = list[pos2], list[pos1] 
    return list


for idx in range(180):
    weights = model.get_layer(index = idx).get_weights()
    if len(weights) == 1: # It is a convolutional layer
        weights = weights[0]
        
        #Storing Convolutional weights
        conv_weights = weights_array[weights_num_counter + 4*weights.shape[-1] : weights_num_counter + np.prod(weights.shape) + 4*weights.shape[-1] ]
        darknet_w_shape = (weights.shape[3], weights.shape[2], weights.shape[0], weights.shape[1])
        conv_weights = np.reshape(conv_weights, darknet_w_shape)
        conv_weights = np.transpose(conv_weights, [2, 3, 1, 0])
        conv_weights = [conv_weights]
        
        #Storing Batch Norm weights
        batch_weights = weights_array[weights_num_counter : weights_num_counter + 4*weights.shape[-1]]
        batch_weights_array = []
        for array in np.array_split(batch_weights, 4):
            batch_weights_array.append(array)
        batch_weights_array = swapPositions(batch_weights_array, 1, 0)

        weights_num_counter += np.prod(weights.shape) + 4*weights.shape[-1]

        model.get_layer(index = idx).set_weights(conv_weights)
        model.get_layer(index = idx + 1).set_weights(batch_weights_array)
        
        
save_path = Path('/home/andrea/AI/ispr_yolo/weights')
save_path = save_path.joinpath('darknet53.h5')
model.save(save_path)
